# Data Extraction

## Parsing Y- axis Ticks and Y- vals 
This code parses the x and y axes (horizontal and vertical axes) detection for the y-ticks, calculates the ratio to normalize the figure contents, specifically the bar plots with respect to the y-ticks and finally connects the evaluated y-values with their correspinding legends using clustering algorithm. 
* Note that all the figures recorded have their respective dois as a unique key of identification.

In [1]:
import cv2, imutils, re
import xlsxwriter, json, os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from matplotlib import rcParams

In [2]:
# Directory of images to run the code on
img_dir = '../AllData/processed_images/Part2'

# Directory to save the output images
save_dir = 'out'

### Using json to parse the captions and dois for each image

In [3]:
with open('../data/urldata.json') as handler:
    urldata = json.load(handler)

In [4]:
with open('../data/doi_data.json') as doihandler:
    doidata = json.load(doihandler)

In [5]:
with open('../data/ocr-image-text.json') as ocrhandler:
    images_text = json.load(ocrhandler)

### Function to detect x-axis and y-axis for a plot

In [6]:
def findMaxConsecutiveOnes(nums) -> int:
    count = maxCount = 0
    
    for i in range(len(nums)):
        if nums[i] == 1:
            count += 1
        else:
            maxCount = max(count, maxCount)
            count = 0
                
    return max(count, maxCount)

In [7]:
def detectAxes(filepath, threshold=None, debug=False):
    if filepath is None:
        return None, None
    
    if threshold is None:
        threshold = 10
    
    image = cv2.imread(filepath)
    height, width, channels = image.shape
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[:, idx] < 200) for idx in range(width)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < width:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            break
            
        start_idx += 1
           
    yaxis = (maxindex, 0, maxindex, height)
    
    if debug:
        fig, ax = plt.subplots(1, 2)

        ax[0].imshow(image)

        ax[1].plot(maxConsecutiveOnes, color = 'k')
        ax[1].axhline(y = max(maxConsecutiveOnes) - 10, color = 'r', linestyle = 'dashed')
        ax[1].axhline(y = max(maxConsecutiveOnes) + 10, color = 'r', linestyle = 'dashed')
        ax[1].vlines(x = maxindex, ymin = 0.0, ymax = maxConsecutiveOnes[maxindex], color = 'b', linewidth = 4)

        plt.show()

    maxConsecutiveOnes = [findMaxConsecutiveOnes(gray[idx, :] < 200) for idx in range(height)]
    start_idx, maxindex, maxcount = 0, 0, max(maxConsecutiveOnes)
    while start_idx < height:
        if abs(maxConsecutiveOnes[start_idx] - maxcount) <= threshold:
            maxindex = start_idx
            
        start_idx += 1
            
    cv2.line(image, (0, maxindex), (width, maxindex),  (255, 0, 0), 2)
    xaxis = (0, maxindex, width, maxindex)
    
    if debug:
        rcParams['figure.figsize'] = 15, 8

        fig, ax = plt.subplots(1, 1)
        ax.imshow(image, aspect = 'auto')
        
    return xaxis, yaxis

In [8]:
def cleanText(image_text):
    
    return [(text, (textx, texty, w, h)) for text, (textx, texty, w, h) in image_text if text.strip() != 'I']

In [9]:
def getProbableLabels(image, image_text, xaxis, yaxis):
    y_labels = []
    x_labels = []
    legends = []
    y_text_list = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    image_text = cleanText(image_text)
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
            
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            numbers = re.findall(r'^[+-]?\d+(?:\.\d+)?[%-]?$', text)
            if bool(numbers):
                y_labels.append((text, (textx, texty, w, h)))
            else:
                y_text_list.append((text, (textx, texty, w, h)))
            
        # To the right of y-axis and bottom of x-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == 1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            x_labels.append((text, (textx, texty, w, h)))
            
        # Top of x-axis and to the right of y-axis
        elif (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == -1):
            
            # Consider non-numeric only for legends
            legends.append((text, (textx, texty, w, h)))
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(y_labels):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(y_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    y_labels_list = maxList.copy()
    
    y_labels = []
    for text, (textx, texty, w, h) in maxList:
        y_labels.append(text)
        
    # Get the x-labels by finding the maximum
    # intersections with the sweeping line
    
    maxIntersection = 0
    maxList = []
    for i in range(y1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_labels):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_labels[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    x_text = x_labels.copy()
    x_labels = []
    hmax = 0
    
    for text, (textx, texty, w, h) in maxList:
        x_labels.append(text)
        if texty + h > hmax:
            hmax = texty + h
    
    # Get possible x-text by moving from where we
    # left off in x-labels to the complete
    # height of the image.
    maxIntersection = 0
    maxList = []
    for i in range(hmax + 1, height):
        count = 0
        current = []
        for index, (text, rect) in enumerate(x_text):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(x_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    x_text = []
    for text, (textx, texty, w, h) in maxList:
        x_text.append(text)
    
    # Get possible legend text
    # For this, we need to search both top to
    # bottom and also from left to right.
    
    legends_and_numbers = mergeTextBoxes(legends)
    
    legends = []
    for text, (textx, texty, w, h) in legends_and_numbers:
        if not re.search(r'^([(+-]*?(\d+)?(?:\.\d+)*?[-%) ]*?)*$', text):
            legends.append((text, (textx, texty, w, h)))
    
    maxIntersection = 0
    maxList = []
    for i in range(y1):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectY(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
            
    for i in range(x11, width):
        count = 0
        current = []
        for index, (text, rect) in enumerate(legends):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(legends[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
        
    legends = []
    for text, (textx, texty, w, h) in maxList:
        legends.append(text)
    
    return image, x_labels, x_text, y_labels, y_labels_list, y_text_list, legends, maxList

In [10]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

## Getting the Ratio for y-value matching 

Similar to the label detection logic, y-ticks are detected: 

Y-ticks 

*  Check only the numerical boxes which are to the left of y-axis and to the top of x-axis.
*  Run a line sweep from left end of the image to the y-axis position, and check when the sweeping line intersects with the maximum number of numerical boxes.
* The numerical boxes are then used as bounding boxes for calculating the y-ticks.

Ratio:
* Difference between the y-ticks is then calculated.  
* Only consider the mean difference between the y-ticks, rejecting the outliers from the calculated values.
* The final ratio is: <br> ``normalize_ratio := ticks_diff.mean() / y-ticks.mean()``.

In [11]:
def getRatio(path, xaxis, yaxis):
    list_text = []
    list_ticks = []
    
    filepath = img_dir + "/" + path.name
    
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height, width, channels = image.shape

    image_text = images_text[path.name]
    
    for text, (textx, texty, w, h) in image_text:
        text = text.strip()
                    
        (x1, y1, x2, y2) = xaxis
        (x11, y11, x22, y22) = yaxis
        
        # To the left of y-axis and top of x-axis
        if (np.sign((x2 - x1) * (texty - y1) - (y2 - y1) * (textx - x1)) == -1 and
            np.sign((x22 - x11) * (texty - y11) - (y22 - y11) * (textx - x11)) == 1):
            
            # Consider numeric only for ticks on y-axis
            numbers = re.findall(r'\d+(?:\.\d+)?', text)
            if bool(numbers):
                list_text.append((numbers[0], (textx, texty, w, h)))
                          
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, (text, rect) in enumerate(list_text):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(list_text[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
    
    # Get list of text and ticks
    list_text = []
    for text, (textx, texty, w, h) in maxList:
        list_text.append(float(text))
        list_ticks.append(float(texty + h))
        
    text_sorted = (sorted(list_text))
    ticks_sorted  = (sorted(list_ticks))
    
    ticks_diff = ([ticks_sorted[i] - ticks_sorted[i-1] for i in range(1, len(ticks_sorted))])
    text_diff = ([text_sorted[i] - text_sorted[i-1] for i in range(1, len(text_sorted))])
    print("[get text-to-tick ratio] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    # Detected text may not be perfect! Remove the outliers.
    ticks_diff = reject_outliers(np.array(ticks_diff), m=1)
    text_diff = reject_outliers(np.array(text_diff), m=1)
    print("[reject_outliers] ticks_diff: {0}, text_diff: {1}".format(ticks_diff, text_diff))
    
    normalize_ratio = np.array(text_diff).mean() / np.array(ticks_diff).mean()

    return text_sorted, normalize_ratio

In [12]:
def lineIntersectsRectX(candx, rect):
    (x, y, w, h) = rect
    
    if x <= candx <= x + w:
        return True
    else:
        return False
    
def lineIntersectsRectY(candy, rect):
    (x, y, w, h) = rect
    
    if y <= candy <= y + h:
        return True
    else:
        return False

In [13]:
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) <= m * np.std(data)]

## Writing to Excel workbook

In [14]:
def addToExcel(dataname, data, row):
    col = 0

    worksheet.write(row, col, dataname)
    for content in data:
        col += 1
        worksheet.write(row, col, content)

## Matching the ratio for final data extraction 

Y-val data:
* The height of each bounding box is calculated by cluster count estimation method. 
* This ratio previously calculated is then used to calculate the <br> ``y_values :=  v_value = normalize_ratio x height of bounding box``.

In [15]:
def mergeRects(contours):
    rects = []
    rectsUsed = []

    # Just initialize bounding rects and set all bools to false
    for cnt in contours:
        rects.append(cv2.boundingRect(cnt))
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[0]

    rects.sort(key = getXFromRect)

    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 5
    yThr = 5

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[0]
            currxMax = supVal[0] + supVal[2]
            curryMin = supVal[1]
            curryMax = supVal[1] + supVal[3]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[0]
                candxMax = subVal[0] + subVal[2]
                candyMin = subVal[1]
                candyMax = subVal[1] + subVal[3]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)

                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currxMin, curryMin, currxMax - currxMin, curryMax - curryMin])
    
    return acceptedRects

In [16]:
def mergeTextBoxes(textboxes):
    rects = []
    rectsUsed = []
    
    # Just initialize bounding rects and set all bools to false
    for box in textboxes:
        rects.append(box)
        rectsUsed.append(False)

    # Sort bounding rects by x coordinate
    def getXFromRect(item):
        return item[1][0]
    
    def getYFromRect(item):
        return item[1][1]

    rects.sort(key = lambda x: (getYFromRect, getXFromRect))
    
    # Array of accepted rects
    acceptedRects = []

    # Merge threshold for x coordinate distance
    xThr = 10
    yThr = 0

    # Iterate all initial bounding rects
    for supIdx, supVal in enumerate(rects):
        if (rectsUsed[supIdx] == False):

            # Initialize current rect
            currxMin = supVal[1][0]
            currxMax = supVal[1][0] + supVal[1][2]
            curryMin = supVal[1][1]
            curryMax = supVal[1][1] + supVal[1][3]
            currText = supVal[0]

            # This bounding rect is used
            rectsUsed[supIdx] = True

            # Iterate all initial bounding rects
            # starting from the next
            for subIdx, subVal in enumerate(rects[(supIdx+1):], start = (supIdx+1)):

                # Initialize merge candidate
                candxMin = subVal[1][0]
                candxMax = subVal[1][0] + subVal[1][2]
                candyMin = subVal[1][1]
                candyMax = subVal[1][1] + subVal[1][3]
                candText = subVal[0]

                # Check if x distance between current rect
                # and merge candidate is small enough
                if (candxMin <= currxMax + xThr):

                    if not nearbyRectangle((candxMin, candyMin, candxMax - candxMin, candyMax - candyMin),
                                           (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin), yThr):
                        break

                    # Reset coordinates of current rect
                    currxMax = candxMax
                    curryMin = min(curryMin, candyMin)
                    curryMax = max(curryMax, candyMax)
                    currText = currText + ' ' + candText
                    
                    # Merge candidate (bounding rect) is used
                    rectsUsed[subIdx] = True
                else:
                    break

            # No more merge candidates possible, accept current rect
            acceptedRects.append([currText, (currxMin, curryMin, currxMax - currxMin, curryMax - curryMin)])
    
    return acceptedRects

In [17]:
def nearbyRectangle(current, candidate, threshold):
    (currx, curry, currw, currh) = current
    (candx, candy, candw, candh) = candidate
    
    currxmin = currx
    currymin = curry
    currxmax = currx + currw
    currymax = curry + currh
    
    candxmin = candx
    candymin = candy
    candxmax = candx + candw
    candymax = candy + candh
    
    # If candidate is on top, and is close
    if candymax <= currymin and candymax + threshold >= currymin:
        return True
    
    # If candidate is on bottom and is close
    if candymin >= currymax and currymax + threshold >= candymin:
        return True
    
    # If intersecting at the top, merge it
    if candymax >= currymin and candymin <= currymin:
        return True
    
    # If intersecting at the bottom, merge it
    if currymax >= candymin and currymin <= candymin:
        return True
    
    # If intersecting on the sides or is inside, merge it
    if (candymin >= currymin and
        candymin <= currymax and
        candymax >= currymin and
        candymax <= currymax):
        return True
    
    return False

In [18]:
def euclidean(v1, v2):
    return sum((p - q) ** 2 for p, q in zip(v1, v2)) ** .5

def angle_between(p1, p2):
    
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]

    return math.atan2(deltaY, deltaX) / math.pi * 180
    
def RectDist(rectA, rectB):
    (rectAx, rectAy, rectAw, rectAh) = rectA
    (rectBx, rectBy, rectBw, rectBh) = rectB
    
    angle = angle_between((rectAx + rectAw / 2, rectAy + rectAh / 2), (rectBx + rectBw / 2, rectBy + rectBh / 2))
    
    if (angle >= -5 and angle <= 5) or (angle <= -175 and angle >= -185) or (angle >= 175 and angle <= 185):
        if (rectBx > rectAx + rectAw):
            return euclidean([rectAx + rectAw / 2, rectAy + rectAh / 2], [rectBx + rectBw / 2, rectBy + rectBh / 2])
        else:
            return sys.maxsize
    else:
        return sys.maxsize

In [19]:
def getProbableYLabels(image, contours, xaxis, yaxis):
    y_labels = []
    
    height, width, channels = image.shape
    
    (x1, y1, x2, y2) = xaxis
    (x11, y11, x22, y22) = yaxis
    
    # Get the y-labels by finding the maximum
    # intersections with the sweeping line
    maxIntersection = 0
    maxList = []
    for i in range(x11):
        count = 0
        current = []
        for index, rect in enumerate(contours):
            if lineIntersectsRectX(i, rect):
                count += 1
                current.append(contours[index])
                            
        if count > maxIntersection:
            maxIntersection = count
            maxList = current
                    
    return image, maxList

## Saving y-values in our data excel sheet 

In [20]:
import ast, cv2, re, sys
import math
import json

images = []
texts = []

def getYVal(img_dir):
    yValueDict = {}
    for index, path in enumerate(Path(img_dir).iterdir()):
        if path.name.endswith('.png') or path.name.endswith('.jpg'):
            filepath = img_dir + "/" + path.name
            img = cv2.imread(filepath)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            xaxis, yaxis = detectAxes(filepath)
            
            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)

            image_text = images_text[path.name]
            img, x_labels, _, _, _, _, legends, legendBoxes = getProbableLabels(img, image_text, xaxis, yaxis)
            try:
                list_text, normalize_ratio = getRatio(path, xaxis, yaxis)
                print("[getYVal] legends: {0}".format(legends))
                print("[{0}] path: {1}, ratio: {2}".format(index, path.name, normalize_ratio), end='\n\n')

                # Color detection
                h, w, _ = np.shape(img)

                image = img.reshape((h * w, 3))

                # Remove white and near-by pixels
                threshold = 5
                for r in range(255 - threshold, 256):
                    for g in range(255 - threshold, 256):
                        for b in range(255 - threshold, 256):
                            image = image[np.where((image != [r, g, b]).any(axis = 1))]

                values, counts = np.unique(image, axis = 0, return_counts = True)

                greater_counts = np.where(counts >= 100)
                values, counts = values[greater_counts], counts[greater_counts]

                sort_indices = np.argsort(-counts)
                values, counts = values[sort_indices], counts[sort_indices]

                groups = []
                groupcounts = []

                for idx, value in enumerate(values):
                    grouped = False

                    for groupid, group in enumerate(groups):
                        for member in group:
                            r, g, b = member
                            vr, vg, vb = value

                            if (abs(vr.astype(np.int16) - r.astype(np.int16)) <= 5 and
                                abs(vg.astype(np.int16) - g.astype(np.int16)) <= 5 and
                                abs(vb.astype(np.int16) - b.astype(np.int16)) <= 5):
                                group.append(value)
                                groupcounts[groupid] += counts[idx]
                                grouped = True
                                break

                        if grouped:
                            break

                    if not grouped:
                        groups.append([value])
                        groupcounts.append(counts[idx])

                groupcounts = np.array(groupcounts)
                sort_indices = np.argsort(-groupcounts)
                new_groups = [groups[i] for i in sort_indices]
                groups = new_groups
                
                for i in range(len(groups[:4])):

                    img = cv2.imread(img_dir + "/" + path.name)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    mask = None
                    for value in groups[i]:
                        COLOR_MIN = np.array([value[0], value[1], value[2]], np.uint8)
                        COLOR_MAX = np.array([value[0], value[1], value[2]], np.uint8)

                        if mask is None:
                            mask = cv2.inRange(img, COLOR_MIN, COLOR_MAX)
                        else:
                            mask = mask | cv2.inRange(img, COLOR_MIN, COLOR_MAX)

                    image = cv2.bitwise_and(img, img, mask = mask)
                    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, (3, 3))

                    edged = cv2.Canny(image, 0, 250)
                    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    contours = [contour for contour in contours if cv2.contourArea(contour) > 0.]

                    # Remove noisy ones!
                    if len(contours) == 0 or len(contours) > 100:
                        continue

                    contours = [cv2.approxPolyDP(contour, 3, True) for contour in contours]

                    rects = mergeRects(contours)
                    textBoxes = []
                    legends = []
                    
                    for rectBox in rects:
                        min_distance = sys.maxsize
                        closestBox = None
                        legendtext = None

                        for text, textBox in legendBoxes:    
                            if RectDist(rectBox, textBox) < min_distance:
                                closestBox = textBox
                                min_distance = RectDist(rectBox, textBox)
                                legendtext = text

                        if closestBox is not None:
                            textBoxes.append(closestBox)

                        if legendtext is not None:
                            legends.append(legendtext)

                    list_len = []
                    
                    for rect in rects:
                        list_len.append(float(rect[3]))

                    # y-values will be a product of the normalize ratio and each length              
                    y_val = [round(l* normalize_ratio, 1) for l in list_len]

                    if path.name in yValueDict:
                        if legends:
                            yValueDict[path.name].append([legends[-1]])
                        else:
                            yValueDict[path.name].append([''])
                            
                        yValueDict[path.name].append(y_val)
                    else:
                        if legends:
                            yValueDict[path.name] = [legends[-1]]
                        else:
                            yValueDict[path.name] = ['']
                            
                        yValueDict[path.name].append(y_val)
                        
                    for box in textBoxes:
                        (x, y, w, h) = box
                        
            except Exception as e:
                print(e)
                continue
            
    return yValueDict

In [21]:
yValueDict = getYVal(img_dir)

[get text-to-tick ratio] ticks_diff: [44.0, 45.0, 45.0, 46.0, 46.0, 44.0], text_diff: [15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
[reject_outliers] ticks_diff: [45. 45.], text_diff: [15. 15. 15. 15. 15. 15.]
[getYVal] legends: ['Aqueous Phase', 'Gaseous Phase', 'Solid Residue']
[0] path: 08813216-Figure1-1.png, ratio: 0.3333333333333333

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['A', 'B']
[1] path: 08813216-Figure4-1.png, ratio: nan



C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\dsand\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\dsand\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:62: RuntimeWar

[get text-to-tick ratio] ticks_diff: [34.0, 34.0, 33.0, 34.0, 35.0, 34.0, 33.0, 35.0, 33.0], text_diff: [0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [34. 34. 34. 34.], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[getYVal] legends: ['SR', 'HSP', 'MSP', 'WSP']
[2] path: 1-s2.0-S0009250913008336-main-Figure10-1.png, ratio: 0.0029411764705882353

[get text-to-tick ratio] ticks_diff: [33.0, 34.0, 31.0, 34.0, 34.0, 34.0, 31.0, 35.0, 32.0], text_diff: [0.1, 0.09999999999999998, 0.10000000000000003, 0.09999999999999998, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998]
[reject_outliers] ticks_diff: [33. 34. 34. 34. 34. 32.], text_diff: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[getYVal] legends: ['0.9-1.0cm', 'SR', 'HSP', 'MSP', 'WSP']
[3] path: 1-s2.0-S0009250913008336-main-Figure11-1.png,

[get text-to-tick ratio] ticks_diff: [71.0, 71.0, 70.0, 72.0, 70.0, 71.0], text_diff: [0.09999999999999998, 0.09999999999999998, 0.10000000000000009, 0.09999999999999998, 0.09999999999999998, 0.10000000000000009]
[reject_outliers] ticks_diff: [71. 71. 71.], text_diff: [0.1 0.1 0.1 0.1]
[getYVal] legends: ['Fuel Specifications:', 'Gasoline:', 'Diesel:', 'Jet']
[24] path: 1-s2.0-S0016236115011084-main-Figure7-1.png, ratio: 0.0014084507042253518

[get text-to-tick ratio] ticks_diff: [65.0, 189.0], text_diff: [60.0, 20.0]
[reject_outliers] ticks_diff: [ 65. 189.], text_diff: [60. 20.]
[getYVal] legends: ['Conversion', 'Yield of paraffin']
[25] path: 1-s2.0-S0016236115011345-main-Figure2-1.png, ratio: 0.31496062992125984

[get text-to-tick ratio] ticks_diff: [70.0, 70.0, 72.0, 69.0], text_diff: [10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [70. 70.], text_diff: [10. 10. 10. 10.]
[getYVal] legends: ['Raw Algae', 'Crude Lipids', 'Residual Solids', 'AC-Purified Lipids', 'K10-Purified 

[get text-to-tick ratio] ticks_diff: [43.0, 41.0, 42.0, 43.0, 42.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [42. 42.], text_diff: [10. 10. 10. 10. 10.]
[getYVal] legends: ['260 C.', "280'C.", '300C']
[47] path: 1-s2.0-S0016236118303363-main-Figure5-1.png, ratio: 0.23809523809523808

[get text-to-tick ratio] ticks_diff: [55.0, 54.0, 55.0, 55.0, 57.0, 53.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [55. 54. 55. 55.], text_diff: [10. 10. 10. 10. 10. 10.]
[getYVal] legends: []
[48] path: 1-s2.0-S0016236118304496-main-Figure4-1.png, ratio: 0.182648401826484

[get text-to-tick ratio] ticks_diff: [51.0, 50.0, 51.0, 51.0, 52.0, 49.0, 51.0, 51.0, 50.0], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [51. 50. 51. 51. 51. 51. 50.], text_diff: [5. 5. 5. 5. 5. 5. 5. 5. 5.]
[getYVal] legends: []
[49] path: 1-s2.0-S0016236118304496-main-Figure5-1.png, ratio: 0.09859154929577464

[get text-to-

...

[get text-to-tick ratio] ticks_diff: [102.0, 101.0, 102.0], text_diff: [0.0, 0.0, 0.0]
[reject_outliers] ticks_diff: [102. 102.], text_diff: [0. 0. 0.]
[getYVal] legends: ['Other*', 'C3H6', 'C2H4', 'CO2', 'CH4', 'H2', '*Other includes: C2H2 and C2H6']
[1137] path: energies-13-00124-v3 (1)-Figure3-1.png, ratio: 0.0

[get text-to-tick ratio] ticks_diff: [62.0, 59.0, 62.0, 61.0, 60.0], text_diff: [0.0, 10.0, 0.0, 10.0, 0.0]
[reject_outliers] ticks_diff: [61. 60.], text_diff: [0. 0. 0.]
[getYVal] legends: ['Other*', 'C3H6', 'C2H4', 'CO2', 'CH4', 'H2', '*Other includes: C2H2 and C2H6']
[1138] path: energies-13-00124-v3 (1)-Figure8-1.png, ratio: 0.0

[get text-to-tick ratio] ticks_diff: [102.0, 101.0, 102.0], text_diff: [0.0, 0.0, 0.0]
[reject_outliers] ticks_diff: [102. 102.], text_diff: [0. 0. 0.]
[getYVal] legends: ['Other*', 'C3H6', 'C2H4', 'CO2', 'CH4', 'H2', '*Other includes: C2H2 and C2H6']
[1139] path: energies-13-00124-v3-Figure3-1.png, ratio: 0.0

[get text-to-tick ratio] ticks_dif

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['Wet route']
[1164] path: Fozer2017_Article_EvaluationOfMicroalgae-basedBi-Figure4-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1165] path: Fozer2017_Article_EvaluationOfMicroalgae-basedBi-Figure5-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [40.0, 41.0], text_diff: [2.0, 2.0]
[reject_outliers] ticks_diff: [40. 41.], text_diff: [2. 2.]
[getYVal] legends: ['b.c', 'T']
[1166] path: gcbb.12171-Figure2-1.png, ratio: 0.04938271604938271

[get text-to-tick ratio] ticks_diff: [53.0, 49.0, 51.0, 51.0, 51.0, 144.0], text_diff: [198.0, 200.0, 200.0, 200.0, 200.0, 200.0]
[reject_outliers] ticks_diff: [53. 49. 51. 51. 51.], text_diff: [200. 200. 200. 200. 200.]
[getYVal] legends: ['Gas plant plant plant nth', 'purchase', 'Pyrolysis', 'Silya produeti', 'electricitul el

[get text-to-tick ratio] ticks_diff: [47.0, 49.0, 47.0, 50.0, 47.0, 48.0], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [47. 49. 47. 47. 48.], text_diff: [5. 5. 5. 5. 5. 5.]
[getYVal] legends: []
[1192] path: Jadhav2019_Article_UtilizationOfOilPalmFrondsForB-Figure6-1.png, ratio: 0.10504201680672269

[get text-to-tick ratio] ticks_diff: [56.0, 116.0, 61.0], text_diff: [20.0, 40.0, 20.0]
[reject_outliers] ticks_diff: [56. 61.], text_diff: [20. 20.]
[getYVal] legends: []
[1193] path: jctb.3933-Figure5-1.png, ratio: 0.3418803418803419

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: ['p=0.05']
[1194] path: Kumar-Pant2016_Article_BiorefinerySolidCakeWasteToBio-Figure2-1.png, ratio: nan

[get text-to-tick ratio] ticks_diff: [], text_diff: []
[reject_outliers] ticks_diff: [], text_diff: []
[getYVal] legends: []
[1195] path: Life cycle analysis of low temperature hydrothermal treatment pathway

[get text-to-tick ratio] ticks_diff: [31.0, 29.0, 34.0, 31.0, 31.0, 31.0, 33.0, 30.0, 32.0, 30.0, 32.0], text_diff: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
[reject_outliers] ticks_diff: [31. 31. 31. 31. 30. 32. 30. 32.], text_diff: [5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
[getYVal] legends: ['GR', '&', 'HGE']
[1217] path: Process modeling synthesis and thermodynamic evaluation of hydrogen production from hydrothermal processing of lipid extracted algae integrated with-Figure5-1.png, ratio: 0.16129032258064516

[get text-to-tick ratio] ticks_diff: [47.0, 49.0, 48.0, 47.0, 49.0], text_diff: [10.0, 10.0, 10.0, 10.0, 10.0]
[reject_outliers] ticks_diff: [48.], text_diff: [10. 10. 10. 10. 10.]
[getYVal] legends: ['Protein Yield', 'Protein Purity', 'Dry Matter', 'Yield/Dissolution']
[1218] path: Pruksasri-Novalin2018_Article_AComprehensiveProcessingOfRice-Figure6-1.png, ratio: 0.20833333333333334

[get text-to-tick ratio] ticks_diff: [44.0, 46.0, 43.0, 91.0], text_diff: [20.0, 10.0,

[get text-to-tick ratio] ticks_diff: [54.0, 48.0, 54.0, 56.0], text_diff: [50.0, 50.0, 50.0, 50.0]
[reject_outliers] ticks_diff: [54. 54. 56.], text_diff: [50. 50. 50. 50.]
[getYVal] legends: ['Residue', 'Oil', 'WS', 'Water', 'Gas']
[1242] path: Sugano2008_Article_HydrothermalLiquefactionOfPlan-Figure5-1.png, ratio: 0.9146341463414634

[get text-to-tick ratio] ticks_diff: [54.0, 69.0, 48.0], text_diff: [20.0, 50.0, 50.0]
[reject_outliers] ticks_diff: [54.], text_diff: [50. 50.]
[getYVal] legends: []
[1243] path: Sugano2009_Article_LiquefactionProcessForAHydroth-Figure8-1.png, ratio: 0.9259259259259259

[get text-to-tick ratio] ticks_diff: [14.0, 42.0, 70.0, 46.0], text_diff: [0.0, 20.0, 50.0, 50.0]
[reject_outliers] ticks_diff: [42. 46.], text_diff: [20. 50. 50.]
[getYVal] legends: ['350 oC- F.B.P.', 'I.B.P- 180', 'oC', 'oC']
[1244] path: Sugano2009_Article_LiquefactionProcessForAHydroth-Figure9-1.png, ratio: 0.9090909090909091

[get text-to-tick ratio] ticks_diff: [30.0, 33.0, 33.0, 3

In [23]:
workbook = xlsxwriter.Workbook('../results/FigureData1.xlsx')

for index, path in enumerate(Path(img_dir).iterdir()):
    if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg'):
        if path.name in yValueDict:
            filepath = img_dir + "/" + path.name
            print("[{0}] path: {1}".format(index, path.name))
            
            image = cv2.imread(filepath)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            height, width, channels = image.shape
            xaxis, yaxis = detectAxes(filepath)
            y_text = []

            for (x1, y1, x2, y2) in [xaxis]:
                xaxis = (x1, y1, x2, y2)

            for (x1, y1, x2, y2) in [yaxis]:
                yaxis = (x1, y1, x2, y2)
                
            image_text = images_text[path.name]
            image, x_labels, x_text, y_labels, y_labels_list, y_text_list, legends, _ = getProbableLabels(image,
                                                                                                  image_text,
                                                                                                  xaxis,
                                                                                                  yaxis)
            
            # Sort bounding rects by y coordinate
            def getYFromRect(item):
                return item[1][1]

            y_labels_list.sort(key = getYFromRect)
            y_text_list.sort(key = getYFromRect, reverse=True)
            
            for text, (textx, texty, w, h) in y_text_list:
                y_text.append(text)

            # Append doi values for each image.
            pdfname = '-'.join(path.name.split('-')[:-2]) + '.pdf'
            
            if pdfname in doidata['Doi']:
                doi = doidata['Doi'][pdfname]
            else:
                doi = ''
             
            # Write to Excel
            worksheet = workbook.add_worksheet()            
            
            addToExcel("doi", doi, 0)
            addToExcel("file name", [path.name], 1)
            addToExcel("x-text", x_text, 2)
            addToExcel("x-labels", x_labels, 3)
            addToExcel("y-text", y_text, 4)
            addToExcel("y-labels", y_labels, 5)
            addToExcel("legends", legends, 6)
            
            for col_num, data in enumerate(yValueDict[path.name]):
                print(col_num, data)
                if type(data) != list:
                    worksheet.write(9 + col_num, 0, "Y-values:" + "\n\n" + str(data))
                else:
                    for pos, d in enumerate(data):
                        worksheet.write(9 + col_num, pos, str(d))

            # Print the output here!
            print("file name    :  ", path.name)
            print("doi          :  ", doi)
            print("x-text       :  ", x_text)
            print("x-labels     :  ", x_labels)
            print("y-text       :  ", y_text)
            print("y-labels     :  ", y_labels)
            print("legends      :  ", legends)
            print("Y- Values    :  ", ' '.join([str(elem) for elem in yValueDict[path.name]]), end = "\n\n")

            # Insert the image
            worksheet.insert_image('J21', filepath)

# Close the excel workbook!
workbook.close()

[0] path: 08813216-Figure1-1.png
0 Aqueous Phase
1 [4.3, 35.0, 0.7, 34.7, 28.7, 30.7, 37.3]
2 ['Solid Residue']
3 [4.7, 26.7, 25.0, 26.3, 22.0, 30.0, 27.0]
4 ['Gaseous Phase']
5 [17.0, 20.7, 25.7, 4.7, 1.7, 20.0, 19.3]
6 ['Solid Residue']
7 [21.0, 18.7, 17.7, 18.3, 15.3, 4.7, 15.3]
file name    :   08813216-Figure1-1.png
doi          :   []
x-text       :   []
x-labels     :   ['270 0', "290'0", '310 C', "330'C", '350']
y-text       :   ['perce', 'ntage', 'of', 'products', '(%)']
y-labels     :   ['105', '90-', '75', '60', '45', '30', '15']
legends      :   ['Aqueous Phase', 'Gaseous Phase', 'Solid Residue']
Y- Values    :   Aqueous Phase [4.3, 35.0, 0.7, 34.7, 28.7, 30.7, 37.3] ['Solid Residue'] [4.7, 26.7, 25.0, 26.3, 22.0, 30.0, 27.0] ['Gaseous Phase'] [17.0, 20.7, 25.7, 4.7, 1.7, 20.0, 19.3] ['Solid Residue'] [21.0, 18.7, 17.7, 18.3, 15.3, 4.7, 15.3]

[1] path: 08813216-Figure4-1.png
0 
1 [nan]
2 ['']
3 [nan, nan]
4 ['B']
5 [nan]
file name    :   08813216-Figure4-1.png
doi         

0 H2
1 [88.6, 2.3, 6.0, 10.9, 2.0, 0.3, 94.9, 94.6, 0.3, 8.9, 0.6, 0.9, 99.7, 2.3, 98.3, 2.0, 100.9, 3.7]
2 ['C02']
3 [0.3, 1.1, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 100.0, 0.3, 1.4, 0.3, 0.3, 0.3, 100.0, 0.3]
4 ['']
5 [91.4, 1.4, 1.1, 94.9]
6 ['']
7 [0.3, 0.3, 0.3, 2.0, 0.3, 70.6, 9.4, 1.7, 0.3, 91.4, 1.7, 2.0, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
file name    :   1-s2.0-S0016236112010988-main-Figure4-1.png
doi          :   ['doi:10.1016/j.fuel.2012.12.074']
x-text       :   ['Water', 'density', '(g/cm3)']
x-labels     :   ['0', '0.013', '0.025', '0.05', '0.075', '0.1']
y-text       :   ['Percentage']
y-labels     :   ['100', '80', '20', '0', '40']
legends      :   ['C2-C5', 'C02', 'CO', 'CH4', 'H2']
Y- Values    :   H2 [88.6, 2.3, 6.0, 10.9, 2.0, 0.3, 94.9, 94.6, 0.3, 8.9, 0.6, 0.9, 99.7, 2.3, 98.3, 2.0, 100.9, 3.7] ['C02'] [0.3, 1.1, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 100.0, 0.3, 1.4, 0.3, 0.3, 0.3, 100.0, 0.3] [''] [91.4, 1.4, 1.1, 94.9] [''

0 
1 [1.9, 3.1, 2.8, 1.3, 2.2, 1.3, 1.9, 1.3]
2 ['Yield of paraffin']
3 [1.3, 0.6, 0.6, 0.6, 0.6, 0.9, 0.6, 0.6, 1.6, 0.6, 0.9, 1.9, 1.6, 1.3, 2.8, 2.2, 1.6, 2.5]
4 ['Yield of paraffin']
5 [0.3, 0.3, 1.6, 0.3, 0.3, 0.6, 0.9, 0.3, 0.3, 0.3, 0.6, 0.3, 0.3, 0.6, 0.3, 0.9, 0.3, 0.3, 0.3, 0.3, 1.9, 0.3, 0.3, 0.3, 0.3, 0.9, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.6, 3.1, 0.3, 0.6, 0.6, 0.3, 0.6, 2.8, 1.6, 0.3, 0.3, 3.1, 1.3, 0.3, 1.6, 0.6, 0.3, 0.3, 1.6, 2.8, 2.5]
file name    :   1-s2.0-S0016236115011345-main-Figure2-1.png
doi          :   ['doi:10.1016/j.fuel.2015.10.120']
x-text       :   ['Temperature', '(C)']
x-labels     :   ['250', '270', '290', '300']
y-text       :   ['Conve', '-rsion', 'PA', 'and', 'yield', 'g', 'of', 'paraffins', '(C', '%)']
y-labels     :   ['100', '80', '20']
legends      :   ['Conversion', 'Yield of paraffin']
Y- Values    :    [1.9, 3.1, 2.8, 1.3, 2.2, 1.3, 1.9, 1.3] ['Yield of paraffin'] [1.3, 0.6, 0.6, 0.6, 0.6, 0.9, 0.6, 0.6, 1.6, 0.6, 0.9, 1.9, 1.6, 1.3, 2.8,

0 Aqueous phase & Gas
1 [0.7, 2.3, 2.9, 5.9, 103.5, 3.6, 1.6, 3.6, 101.6, 2.9, 104.8, 1.0, 2.3, 3.3, 2.6, 2.3, 104.8, 2.3, 3.9, 4.2, 101.9, 3.3, 1.6, 2.6, 101.6, 2.0, 47.3, 2.6, 2.6, 11.1, 16.7, 2.0, 0.3, 1.6, 3.3, 10.1, 103.2]
file name    :   1-s2.0-S0016236117301709-main-Figure5-1.png
doi          :   ['doi:10.1016/j.fuel.2017.02.022']
x-text       :   []
x-labels     :   ['A', 'D', 'Sample B', 'Sample F', 'Sample G', 'sampleg', 'Sample', 'Sample']
y-text       :   ['yier']
y-labels     :   ['120%', '110%', '100%', '90%', '80%', '60%', '40%', '30%', '20%', '10%', '0%']
legends      :   ['Bio-crude oil', 'Solid residue', 'Aqueous phase & Gas']
Y- Values    :   Aqueous phase & Gas [0.7, 2.3, 2.9, 5.9, 103.5, 3.6, 1.6, 3.6, 101.6, 2.9, 104.8, 1.0, 2.3, 3.3, 2.6, 2.3, 104.8, 2.3, 3.9, 4.2, 101.9, 3.3, 1.6, 2.6, 101.6, 2.0, 47.3, 2.6, 2.6, 11.1, 16.7, 2.0, 0.3, 1.6, 3.3, 10.1, 103.2]

[35] path: 1-s2.0-S0016236117305367-main-Figure4-1.png
0 
1 [0.2, 0.3, 0.2, 0.2, 27.4, 0.7, 0.7, 4.7, 0.

0 Gas,
1 [11.5, 11.7, 3.3, 8.8]
2 ['Bio-residue']
3 [0.2, 0.2, 0.2, 0.4, 0.2, 1.2, 0.2, 0.2, 0.2, 1.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 2.1, 0.2, 0.2, 0.6, 0.2, 0.2, 0.2, 0.2, 0.2, 66.5, 0.2, 1.7, 0.2, 1.0, 0.2, 0.2, 2.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.2, 0.2, 0.2, 0.2, 1.2, 0.2, 2.1, 0.2, 0.2, 1.2, 0.2, 0.2, 1.0, 0.2, 0.4, 0.4, 0.2, 0.2, 0.2, 1.2]
4 ['']
5 [0.4, 3.8, 1.0, 1.3, 1.7, 1.0, 1.7, 1.3]
file name    :   1-s2.0-S0016236118303363-main-Figure3-1.png
doi          :   ['doi:10.1016/j.fuel.2018.02.153']
x-text       :   ['Hydrothermal', 'Temperature']
x-labels     :   ['260', '280', '300']
y-text       :   ['rouCTO', 'yield,', '%']
y-labels     :   ['60', '50', '40', '20', '10', '30']
legends      :   ['Bio-oil,', 'Gas,', 'Bio-residue']
Y- Values    :   Gas, [11.5, 11.7, 3.3, 8.8] ['Bio-residue'] [0.2, 0.2, 0.2, 0.4, 0.2, 1.2, 0.2, 0.2, 0.2, 1.0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 2.1, 0.2, 0.2, 0.6, 0.2, 0.2, 0.2, 0.2, 0.2, 66.5, 0.2, 1.7, 0.2, 1.0, 0.2, 0.2, 2.3, 0.

...

0 Na2C03
1 [4.7, 4.3, 3.4, 4.2, 0.2]
2 ['NaOH']
3 [4.0, 4.8, 2.5, 3.5, 0.2]
4 ['NaOH']
5 [3.5, 2.9, 2.8, 2.8, 0.2]
6 ['Na2C03']
7 [0.1, 0.1, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 6.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.0, 0.1, 0.0, 0.2, 0.1, 0.1, 0.1, 0.1, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.1, 0.2, 0.0]
file name    :   Midgett2012_Article_AssessingFeedstocksAndCatalyst-Figure3-1.png
doi          :   []
x-text       :   ['Catalyst', '(g)']
x-labels     :   ['2', '3', '4']
y-text       :   ['Acetone', 'Soluble', "F'rafons", 'g']
y-labels     :   ['4', '3', '5', '2']
legends      :   ['No Catalyst', 'Na2C03', 'K2C03', 'NaOH']
Y- Values    :   Na2C03 [4.7, 4.3, 3.4, 4.2, 0.2] ['NaOH'] [4.0, 4.8, 2.5, 3.5, 0.2] ['NaOH'] [3.5, 2.9, 2.8, 2.8, 0.2] ['Na2C03'] [0.1, 0.1, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 

0 
1 [8.1, 12.3, 15.0, 21.3]
file name    :   Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.10-1.png
doi          :   []
x-text       :   ['Carbon', 'Activated', 'Carbon', 'Charcoal', 'Charcoal']
x-labels     :   ['Coal', 'Activated', 'Coconut', 'Shell', 'Macadamia', 'Shell', 'Spruce', 'wood']
y-text       :   ['Hydrogen', 'Yield,', 'mol/kg']
y-labels     :   ['25', '20', '15', '10', '5']
legends      :   []
Y- Values    :    [8.1, 12.3, 15.0, 21.3]

[1208] path: Onwudili2014_Chapter_HydrothermalGasificationOfBiom-Figure10.11-1.png
0 
1 [0.1, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.4, 2.4, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 12.5, 0.3, 0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 1.0, 0.1, 0.5, 10.9, 0.1, 0.3, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 31.2, 0.4, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 15.5, 0.1, 0.1, 0.5, 0.1, 0.1, 0.1, 0.

0 
1 [33.4, 38.5, 34.4, 34.4, 38.5, 26.3, 42.5, 51.6, 50.6, 45.1, 17.7, 24.3, 1.0]
2 ['']
3 [20.3, 12.2, 16.2, 26.3, 16.2, 35.9, 20.3, 26.8, 22.8, 28.4, 28.4, 24.3, 1.0]
4 ['']
5 [19.2, 24.3, 26.3, 14.2, 20.3, 22.3, 18.2, 8.1, 6.1, 8.1, 38.5, 36.5]
file name    :   s12866-017-1144-x-Figure4-1.png
doi          :   ['doi:10.1186/s12866-017-1144-x']
x-text       :   ['To theri', 'indal.', 'eochtoo']
x-labels     :   ['TGA', 'TGAR', 'TGA', 'TGAS', 'PG', 'PBGA', 'REEL', 'PBGA,', 'PGA', 'REP,', 'Sp.', 'SD', 'Sp.', 'sp.', 'chor', 'oabund', 'atori']
y-text       :   []
y-labels     :   ['120', '100', '80']
legends      :   []
Y- Values    :    [33.4, 38.5, 34.4, 34.4, 38.5, 26.3, 42.5, 51.6, 50.6, 45.1, 17.7, 24.3, 1.0] [''] [20.3, 12.2, 16.2, 26.3, 16.2, 35.9, 20.3, 26.8, 22.8, 28.4, 28.4, 24.3, 1.0] [''] [19.2, 24.3, 26.3, 14.2, 20.3, 22.3, 18.2, 8.1, 6.1, 8.1, 38.5, 36.5]

[1223] path: s40643-018-0233-5-Figure3-1.png
0 
1 [3.2, 2.2, 17.2, 4.5, 6.0, 9.8, 5.2, 10.2, 5.5, 5.8, 7.8, 8.5, 2.0, 6

0 nitro-compounds
1 [1.8, 0.5, 0.2, 17.7, 2.7, 0.2, 0.3, 0.5, 0.2, 1.4, 0.5, 0.5, 0.6, 0.2, 0.3, 0.5, 0.8, 0.8, 0.2, 0.6, 2.1, 1.9, 1.1, 16.6, 0.3, 0.2, 13.4, 0.2, 1.0, 1.6, 1.4, 0.8, 1.6, 7.0, 1.1, 1.0, 0.6, 1.1, 1.1, 8.0, 16.6, 1.0, 2.2, 1.0, 0.5, 0.3, 0.2, 0.2, 1.3, 0.6, 0.3, 0.2, 1.3, 0.5]
file name    :   slct.201601903-Figure2-1.png
doi          :   []
x-text       :   ['SPV/LJ', 'mass', 'ratio']
x-labels     :   ['0/100', '1:3', '1:2', '1:1', '2:1', '3:1', '100/0']
y-text       :   ['Relative', 'content', '(%)']
y-labels     :   ['50', '10', '40', '30', '20']
legends      :   ['aromatics', 'phenols', 'alkanes and alkenes', 'other oxy-compounds', 'nitro-compounds']
Y- Values    :   nitro-compounds [1.8, 0.5, 0.2, 17.7, 2.7, 0.2, 0.3, 0.5, 0.2, 1.4, 0.5, 0.5, 0.6, 0.2, 0.3, 0.5, 0.8, 0.8, 0.2, 0.6, 2.1, 1.9, 1.1, 16.6, 0.3, 0.2, 13.4, 0.2, 1.0, 1.6, 1.4, 0.8, 1.6, 7.0, 1.1, 1.0, 0.6, 1.1, 1.1, 8.0, 16.6, 1.0, 2.2, 1.0, 0.5, 0.3, 0.2, 0.2, 1.3, 0.6, 0.3, 0.2, 1.3, 0.5]

[1239] path

0 
1 [5.2, 1.0, 17.2, 1.0]
2 ['']
3 [7.5, 4.8, 10.3, 1.0, 1.2]
file name    :   Wagner2018_Article_HydrothermalConversionOfLipid--Figure2-1.png
doi          :   ['doi:10.1007/s12649-017-9944-7']
x-text       :   []
x-labels     :   ['vol%', 'IPA', '50', 'vol%', 'IPA']
y-text       :   ['BIOCI', 'apn-', 'yie', 'd', '(%)']
y-labels     :   ['35%', '30%', '25%', '20%', '15%', '10%', '5%', '0%']
legends      :   ['Light Oil', 'Heavy Oil']
Y- Values    :    [5.2, 1.0, 17.2, 1.0] [''] [7.5, 4.8, 10.3, 1.0, 1.2]

[1250] path: wene.319-Figure5-1.png
0 
1 [47.8, 6.7, 50.0, 52.2]
2 ['']
3 [37.8, 40.6, 6.7, 41.7]
4 ['']
5 [6.7, 14.4, 14.4, 14.4]
file name    :   wene.319-Figure5-1.png
doi          :   []
x-text       :   []
x-labels     :   ['LLE', 'HDO', 'reators', 'Chemical', 'storage']
y-text       :   ['SM', 'Wi..', 'installed']
y-labels     :   ['140', '120', '100', '80', '60', '40', '20']
legends      :   ['capital costs']
Y- Values    :    [47.8, 6.7, 50.0, 52.2] [''] [37.8, 40.6, 6.7, 41.